In [1]:
import sys, os
sys.path.append(os.path.abspath("../"))  
import pandas as pd
from core.factors import *

In [3]:
df = pd.read_parquet("../data/taifex_stock.parquet")

In [4]:
high = df.pivot(index="date", columns="stock_id", values="high").sort_index()
low  = df.pivot(index="date", columns="stock_id", values="low").sort_index()
volume = df.pivot(index="date", columns="stock_id", values="volume").sort_index()
close = df.pivot(index="date", columns="stock_id", values="close").sort_index()
open = df.pivot(index="date", columns="stock_id", values="open").sort_index()

In [5]:
def apply_liquidity_filter(factor: pd.DataFrame,
                           vol: pd.DataFrame,
                           close: pd.DataFrame,
                           top_n: int = 100) -> pd.DataFrame:
    """Keep only top-N stocks by (volume × close) each day."""
    dollar_vol = vol * close
    ranks = dollar_vol.rank(axis=1, ascending=False, method="first")
    mask = ranks <= top_n
    return factor.where(mask)


In [6]:
from core.factors import *

# close: DataFrame (date×asset)
delta1 = ts_delta(close, 1)
tsmin4 = ts_min(delta1, 4)
tsmax4 = ts_max(delta1, 4)

# condition logic
cond1 = greater(tsmin4, 0)
cond2 = less(tsmax4, 0)

# nested ternary condition
temp = where(cond1, delta1, where(cond2, delta1, mul(-1, delta1)))

# final cross-sectional rank
alpha10 = rank_cs(temp)

In [7]:
rank_high = rank_cs(high)
rank_vol  = rank_cs(volume)

corr_3d = corr_ts(rank_high, rank_vol, d=3)

rank_corr = rank_cs(corr_3d)

sum3 = ts_sum(rank_corr, d=3)

alpha15 = -1 * sum3

NameError: name 'vol' is not defined

In [11]:
amount = volume*close                        

d10 = ts_delta(close, 10)
r1  = rank_cs(d10)
r2  = rank_cs(r1)
inner = -1 * r2

# decay_linear(..., 10) then triple rank
dec  = decay_linear(inner, d=10)
A = rank_cs(rank_cs(rank_cs(dec)))

# ---------- part B ----------
# rank((-1) * delta(close, 3))
d3 = ts_delta(close, 3)
B  = rank_cs(-1 * d3)

# ---------- part C ----------
# sign(scale(correlation(adv20, low, 12)))
adv20 = adv(amount, d=20)
corr12 = corr_ts(adv20, low, d=12)
C = np.sign(scale_cs(corr12))  # sign after cross-sectional scale

# ---------- alpha31 ----------
alpha31 = A + B + C

In [12]:
intraday_ret = (close - open) / open.replace(0, np.nan)

alpha2 = -1 * corr_ts(
    rank_cs(ts_delta(log(volume), 2)),   # rank(delta(log(volume), 2))
    rank_cs(intraday_ret),               # rank((close-open)/open)
    d=6                                  # correlation over 6 days
)

In [13]:
alpha40 = (-1*rank_cs(ts_std(high,10)))*corr_ts(high,volume,10)

In [14]:
vwap    = amount / volume                          


In [15]:

alpha5 = rank_cs(open - ts_sum(vwap, 10) / 10) * (-1 * abs_(rank_cs(close - vwap)))


In [18]:
import pandas as pd
from core.backtest import backtest_core
# your factor (date×asset), expected returns matrix (date×asset)
# e.g. expected return = next-day close-to-close return
ret = close.pct_change()          # target return
factor =normalize(alpha31)+normalize(alpha10)

/tmp/ipykernel_42147/3094606384.py:5: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret = close.pct_change()          # target return


In [19]:
import pandas as pd

df_sector = pd.read_parquet("../data/twse_2020_to_2025_sector.parquet")
sector_map = df_sector.drop_duplicates(subset=["stock_id"])[["stock_id", "sector"]]
sector_map = sector_map.set_index("stock_id")["sector"]


In [21]:
factor_liquid = apply_liquidity_filter(alpha31, volume, close, top_n=100)


In [22]:
factor =normalize(alpha31)+normalize(alpha10)

In [23]:
alpha_158_1  = (close - open) / open
alpha_158_2  = (high - low) / open
alpha_158_3  = (close - open) / (high - low + 1e-12)
alpha_158_4  = (high - np.maximum(open, close)) / open
alpha_158_5  = (high - np.maximum(open, close)) / (high - low + 1e-12)
alpha_158_6  = (np.minimum(open, close) - low) / open
alpha_158_7  = (np.minimum(open, close) - low) / (high - low + 1e-12)
alpha_158_8  = (2 * close - high - low) / open
alpha_158_9  = (2 * close - high - low) / (high - low + 1e-12)
alpha_158_10 = open / close

In [24]:
vwap    = amount / volume                          

alpha_158_11 = (close - low) / (high - low + 1e-12)
alpha_158_12 = (close - open) / (high - open + 1e-12)
alpha_158_13 = (open - low) / (high - low + 1e-12)
alpha_158_14 = (high - close) / (high - low + 1e-12)

alpha_158_15 = open - ts_sum(vwap, 10) / 10

alpha_158_16 = close - vwap
alpha_158_17 = close - open
alpha_158_18 = ts_delta(close, 1)
alpha_158_19 = ts_delta(close, 5)
alpha_158_20 = ts_delta(close, 10)


In [25]:
alpha_158_21 = ts_delta(close, 20)
alpha_158_22 = ts_delta(close, 60)
alpha_158_23 = ts_delta(vwap, 1)
alpha_158_24 = ts_delta(vwap, 5)
alpha_158_25 = ts_delta(vwap, 10)
alpha_158_26 = ts_delta(vwap, 20)
alpha_158_27 = ts_delta(vwap, 60)
alpha_158_28 = close / ts_mean(close, 5)
alpha_158_29 = close / ts_mean(close, 10)
alpha_158_30 = close / ts_mean(close, 20)


In [26]:
alpha_158_31 = close / ts_mean(close, 60)
alpha_158_32 = vwap / ts_mean(vwap, 5)
alpha_158_33 = vwap / ts_mean(vwap, 10)
alpha_158_34 = vwap / ts_mean(vwap, 20)
alpha_158_35 = vwap / ts_mean(vwap, 60)
alpha_158_36 = ts_std(close, 5)
alpha_158_37 = ts_std(close, 10)
alpha_158_38 = ts_std(close, 20)
alpha_158_39 = ts_std(close, 60)
alpha_158_40 = ts_std(vwap, 5)


In [27]:
alpha_158_41 = ts_std(vwap, 10)
alpha_158_42 = ts_std(vwap, 20)
alpha_158_43 = ts_std(vwap, 60)
alpha_158_44 = ts_max(high, 5) - ts_min(low, 5)
alpha_158_45 = ts_max(high, 10) - ts_min(low, 10)
alpha_158_46 = ts_max(high, 20) - ts_min(low, 20)
alpha_158_47 = ts_max(high, 60) - ts_min(low, 60)
alpha_158_48 = close / ts_max(close, 5)
alpha_158_49 = close / ts_max(close, 10)
alpha_158_50 = close / ts_max(close, 20)


In [28]:
alpha_158_51 = close / ts_max(close, 60)
alpha_158_52 = close / ts_min(close, 5)
alpha_158_53 = close / ts_min(close, 10)
alpha_158_54 = close / ts_min(close, 20)
alpha_158_55 = close / ts_min(close, 60)
alpha_158_56 = vwap / ts_max(vwap, 5)
alpha_158_57 = vwap / ts_max(vwap, 10)
alpha_158_58 = vwap / ts_max(vwap, 20)
alpha_158_59 = vwap / ts_max(vwap, 60)
alpha_158_60 = vwap / ts_min(vwap, 5)


In [29]:
alpha_158_61 = vwap / ts_min(vwap, 10)
alpha_158_62 = vwap / ts_min(vwap, 20)
alpha_158_63 = vwap / ts_min(vwap, 60)
alpha_158_64 = ts_mean(close, 5) / ts_std(close, 5)
alpha_158_65 = ts_mean(close, 10) / ts_std(close, 10)
alpha_158_66 = ts_mean(close, 20) / ts_std(close, 20)
alpha_158_67 = ts_mean(close, 60) / ts_std(close, 60)
alpha_158_68 = ts_mean(vwap, 5) / ts_std(vwap, 5)
alpha_158_69 = ts_mean(vwap, 10) / ts_std(vwap, 10)
alpha_158_70 = ts_mean(vwap, 20) / ts_std(vwap, 20)


In [30]:
alpha_158_71 = ts_mean(vwap, 60) / ts_std(vwap, 60)
alpha_158_72 = volume / ts_mean(volume, 5)
alpha_158_73 = volume / ts_mean(volume, 10)
alpha_158_74 = volume / ts_mean(volume, 20)
alpha_158_75 = volume / ts_mean(volume, 60)
alpha_158_76 = ts_delta(volume, 1)
alpha_158_77 = ts_delta(volume, 5)
alpha_158_78 = ts_delta(volume, 10)
alpha_158_79 = ts_delta(volume, 20)
alpha_158_80 = ts_delta(volume, 60)


In [31]:
alpha_158_81 = ts_mean(volume, 5)
alpha_158_82 = ts_mean(volume, 10)
alpha_158_83 = ts_mean(volume, 20)
alpha_158_84 = ts_mean(volume, 60)
alpha_158_85 = ts_std(volume, 5)
alpha_158_86 = ts_std(volume, 10)
alpha_158_87 = ts_std(volume, 20)
alpha_158_88 = ts_std(volume, 60)
alpha_158_89 = amount / ts_mean(amount, 5)
alpha_158_90 = amount / ts_mean(amount, 10)


In [32]:
alpha_158_91  = amount / ts_mean(amount, 20)
alpha_158_92  = amount / ts_mean(amount, 60)
alpha_158_93  = ts_delta(amount, 1)
alpha_158_94  = ts_delta(amount, 5)
alpha_158_95  = ts_delta(amount, 10)
alpha_158_96  = ts_delta(amount, 20)
alpha_158_97  = ts_delta(amount, 60)
alpha_158_98  = ts_std(amount, 5)
alpha_158_99  = ts_std(amount, 10)
alpha_158_100 = ts_std(amount, 20)


In [33]:
alpha_158_101 = ts_std(amount, 60)
alpha_158_102 = ts_mean(high - low, 5)
alpha_158_103 = ts_mean(high - low, 10)
alpha_158_104 = ts_mean(high - low, 20)
alpha_158_105 = ts_mean(high - low, 60)
alpha_158_106 = ts_std(high - low, 5)
alpha_158_107 = ts_std(high - low, 10)
alpha_158_108 = ts_std(high - low, 20)
alpha_158_109 = ts_std(high - low, 60)
alpha_158_110 = (close - ts_mean(close, 5)) / (ts_std(close, 5) + 1e-12)


In [34]:
alpha_158_111 = (close - ts_mean(close, 10)) / (ts_std(close, 10) + 1e-12)
alpha_158_112 = (close - ts_mean(close, 20)) / (ts_std(close, 20) + 1e-12)
alpha_158_113 = (close - ts_mean(close, 60)) / (ts_std(close, 60) + 1e-12)

alpha_158_114 = (vwap - ts_mean(vwap, 5)) / (ts_std(vwap, 5) + 1e-12)
alpha_158_115 = (vwap - ts_mean(vwap, 10)) / (ts_std(vwap, 10) + 1e-12)
alpha_158_116 = (vwap - ts_mean(vwap, 20)) / (ts_std(vwap, 20) + 1e-12)
alpha_158_117 = (vwap - ts_mean(vwap, 60)) / (ts_std(vwap, 60) + 1e-12)

alpha_158_118 = (ts_mean(close, 5) - ts_mean(close, 20)) / (ts_std(close, 20) + 1e-12)
alpha_158_119 = (ts_mean(close, 10) - ts_mean(close, 60)) / (ts_std(close, 60) + 1e-12)
alpha_158_120 = (ts_mean(vwap, 5) - ts_mean(vwap, 20)) / (ts_std(vwap, 20) + 1e-12)


In [35]:
alpha_158_121 = (ts_mean(vwap, 10) - ts_mean(vwap, 60)) / (ts_std(vwap, 60) + 1e-12)
alpha_158_122 = (ts_mean(volume, 5) - ts_mean(volume, 20)) / (ts_std(volume, 20) + 1e-12)
alpha_158_123 = (ts_mean(volume, 10) - ts_mean(volume, 60)) / (ts_std(volume, 60) + 1e-12)

alpha_158_124 = ts_max(close, 5) - ts_min(close, 5)
alpha_158_125 = ts_max(close, 10) - ts_min(close, 10)
alpha_158_126 = ts_max(close, 20) - ts_min(close, 20)
alpha_158_127 = ts_max(close, 60) - ts_min(close, 60)

alpha_158_128 = ts_max(vwap, 5) - ts_min(vwap, 5)
alpha_158_129 = ts_max(vwap, 10) - ts_min(vwap, 10)
alpha_158_130 = ts_max(vwap, 20) - ts_min(vwap, 20)


In [36]:
alpha_158_131 = ts_max(vwap, 60) - ts_min(vwap, 60)

alpha_158_132 = ts_mean(high, 5) - ts_mean(low, 5)
alpha_158_133 = ts_mean(high, 10) - ts_mean(low, 10)
alpha_158_134 = ts_mean(high, 20) - ts_mean(low, 20)
alpha_158_135 = ts_mean(high, 60) - ts_mean(low, 60)

alpha_158_136 = ts_std(high, 5) - ts_std(low, 5)
alpha_158_137 = ts_std(high, 10) - ts_std(low, 10)
alpha_158_138 = ts_std(high, 20) - ts_std(low, 20)
alpha_158_139 = ts_std(high, 60) - ts_std(low, 60)

alpha_158_140 = (close - ts_min(close, 5)) / (ts_max(close, 5) - ts_min(close, 5) + 1e-12)


In [37]:
alpha_158_141 = (close - ts_min(close, 10)) / (ts_max(close, 10) - ts_min(close, 10) + 1e-12)
alpha_158_142 = (close - ts_min(close, 20)) / (ts_max(close, 20) - ts_min(close, 20) + 1e-12)
alpha_158_143 = (close - ts_min(close, 60)) / (ts_max(close, 60) - ts_min(close, 60) + 1e-12)

alpha_158_144 = (vwap - ts_min(vwap, 5)) / (ts_max(vwap, 5) - ts_min(vwap, 5) + 1e-12)
alpha_158_145 = (vwap - ts_min(vwap, 10)) / (ts_max(vwap, 10) - ts_min(vwap, 10) + 1e-12)
alpha_158_146 = (vwap - ts_min(vwap, 20)) / (ts_max(vwap, 20) - ts_min(vwap, 20) + 1e-12)
alpha_158_147 = (vwap - ts_min(vwap, 60)) / (ts_max(vwap, 60) - ts_min(vwap, 60) + 1e-12)

alpha_158_148 = (volume - ts_min(volume, 5)) / (ts_max(volume, 5) - ts_min(volume, 5) + 1e-12)
alpha_158_149 = (volume - ts_min(volume, 10)) / (ts_max(volume, 10) - ts_min(volume, 10) + 1e-12)
alpha_158_150 = (volume - ts_min(volume, 20)) / (ts_max(volume, 20) - ts_min(volume, 20) + 1e-12)


In [38]:
alpha_158_151 = (volume - ts_min(volume, 60)) / (ts_max(volume, 60) - ts_min(volume, 60) + 1e-12)

alpha_158_152 = (amount - ts_min(amount, 5)) / (ts_max(amount, 5) - ts_min(amount, 5) + 1e-12)
alpha_158_153 = (amount - ts_min(amount, 10)) / (ts_max(amount, 10) - ts_min(amount, 10) + 1e-12)
alpha_158_154 = (amount - ts_min(amount, 20)) / (ts_max(amount, 20) - ts_min(amount, 20) + 1e-12)
alpha_158_155 = (amount - ts_min(amount, 60)) / (ts_max(amount, 60) - ts_min(amount, 60) + 1e-12)

alpha_158_156 = ts_mean(close - open, 5)
alpha_158_157 = ts_mean(close - open, 10)
alpha_158_158 = ts_mean(close - open, 20)


In [44]:

bm = ret.mean(axis=1)                       # simple EW market as benchmark


report = backtest_core(
    factor=-alpha_158_9,
    exp_ret=ret,
    period_of_year=252,
    benchmark=bm,
    fee=0.0001,
    exec_lag=1,                 
    sector_map=sector_map,
    neutralize=True,           # turn on if you pass sector_map
    preprocess='rank', # or "rank" / "none"
    leverage=1.0,
    cap=0.1,                   # per-name cap (optional)
    return_weights=True,
    plot=True,
)

print(report.summary)
turnover = report.weights.pipe(lambda w: (w.diff().abs().sum(axis=1)).mean())
print("Avg turnover:", round(float(turnover), 6))


/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.



                      Performance Benchmark
Cumprod Total Returns    230.37 %  105.77 %
Cumsum Total Returns     120.00 %   83.40 %
Sharpe Ratio                 5.43      0.76
Sortino Ratio                7.85      0.83
Annualized Ret            24.37 %   14.08 %
Max Drawdown               3.19 %   33.40 %
Volatility                 4.03 %   20.15 %
STD                        0.25 %    1.27 %
Avg turnover: 1.3394


In [112]:
df

,market,stock_id,date,open,high,low,close,volume,sector
0,TWSE,1101,2020-01-02,43.80,44.15,43.80,44.10,18470566,01
1,TWSE,1101,2020-01-03,44.15,44.25,43.45,43.95,18387114,01
2,TWSE,1101,2020-01-06,43.50,43.70,43.30,43.45,13867625,01
3,TWSE,1101,2020-01-07,43.45,43.60,43.20,43.60,14185347,01
4,TWSE,1101,2020-01-08,43.15,43.55,43.15,43.40,13465670,01
...,...,...,...,...,...,...,...,...,...
1319592,TWSE,9958,2025-09-01,183.50,185.00,175.00,178.00,3488965,10
1319593,TWSE,9958,2025-09-02,178.50,179.50,175.00,176.00,2021527,10
1319594,TWSE,9958,2025-09-03,176.50,177.50,173.00,173.00,1930029,10
1319595,TWSE,9958,2025-09-04,174.50,176.50,173.50,173.50,1379901,10


In [216]:
alpha_dict = {k: globals()[k] for k in globals() if k.startswith("alpha_158_")}
alpha_dict = dict(sorted(alpha_dict.items(), key=lambda x: int(x[0].split("_")[-1])))


In [221]:
report.summary


,Performance,Benchmark
Cumprod Total Returns,-11.09 %,105.77 %
Cumsum Total Returns,-11.05 %,83.40 %
Sharpe Ratio,-0.40,0.76
Sortino Ratio,-0.61,0.83
Annualized Ret,-2.12 %,14.08 %
Max Drawdown,18.53 %,33.40 %
Volatility,5.08 %,20.15 %
STD,0.32 %,1.27 %


In [222]:
results = []

for name, factor in alpha_dict.items():
    print("Running", name)

    try:
        report = backtest_core(
            factor=factor,
            exp_ret=ret,
            period_of_year=252,
            benchmark=bm,
            fee=0.000,
            exec_lag=1,
            sector_map=sector_map,
            neutralize=True,
            preprocess='rank',
            leverage=1.0,
            cap=0.1,
            return_weights=False,
            plot=False,
        )

        sr = float(report.summary.loc['Sortino Ratio', 'Performance'])
        results.append({"factor": name, "sortino": sr})

    except Exception as e:
        print(f"Error on {name}: {e}")


Running alpha_158_1
Running alpha_158_2
Running alpha_158_3
Running alpha_158_4
Running alpha_158_5
Running alpha_158_6
Running alpha_158_7
Running alpha_158_8
Running alpha_158_9


/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.grou

Running alpha_158_10
Running alpha_158_11
Running alpha_158_12
Running alpha_158_13
Running alpha_158_14
Running alpha_158_15
Running alpha_158_16
Running alpha_158_17
Running alpha_158_18
Running alpha_158_19


/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.grou

Running alpha_158_20
Running alpha_158_21
Running alpha_158_22
Running alpha_158_23
Running alpha_158_24
Running alpha_158_25
Running alpha_158_26
Running alpha_158_27
Running alpha_158_28
Running alpha_158_29


...

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.grou

Running alpha_158_144
Running alpha_158_145
Running alpha_158_146
Running alpha_158_147
Running alpha_158_148
Running alpha_158_149
Running alpha_158_150
Running alpha_158_151
Running alpha_158_152
Running alpha_158_153


/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.grou

Running alpha_158_154
Running alpha_158_155
Running alpha_158_156
Running alpha_158_157
Running alpha_158_158


/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.



In [223]:
df_sr = pd.DataFrame(results).dropna()
df_sr_sorted = df_sr.sort_values("sortino", ascending=False)

top10 = df_sr_sorted.head(10)
bottom10 = df_sr_sorted.tail(10)


In [224]:
top10

,factor,sortino
13,alpha_158_14,8.57
4,alpha_158_5,7.73
3,alpha_158_4,6.38
9,alpha_158_10,3.77
54,alpha_158_55,1.58
62,alpha_158_63,1.57
1,alpha_158_2,1.43
41,alpha_158_42,1.20
37,alpha_158_38,1.20
129,alpha_158_130,1.17


In [225]:
bottom10

,factor,sortino
17,alpha_158_18,-5.18
22,alpha_158_23,-5.19
0,alpha_158_1,-5.32
11,alpha_158_12,-5.34
2,alpha_158_3,-5.63
16,alpha_158_17,-6.38
6,alpha_158_7,-7.12
7,alpha_158_8,-7.99
10,alpha_158_11,-8.87
8,alpha_158_9,-9.32


In [231]:
df_sr_sorted

,factor,sortino
13,alpha_158_14,8.57
4,alpha_158_5,7.73
3,alpha_158_4,6.38
9,alpha_158_10,3.77
54,alpha_158_55,1.58
...,...,...
16,alpha_158_17,-6.38
6,alpha_158_7,-7.12
7,alpha_158_8,-7.99
10,alpha_158_11,-8.87
